In [1]:
import win32com.client as win32
from PIL import ImageGrab
import os
import time

In [2]:
html="""
    <p>
    Buen día equipo,
    </p>
    <p>A continuación envio el reporte de variaciones en las órdenes de producción, cuya variación actual suma
    <b>{:,.0f} MM COP.</p>
    <p align="center"><img src="cid:{}" alt="Tabla Descripción generada automáticamente"></p>
    <p>
    El link del reporte es el siguiente: 
    <a href="https://grupoexito-my.sharepoint.com/:x:/g/personal/jcleiva_grupo-exito_com/EWAKvrk8vGxHkpzJWNin06IBMwDw6KwLhErMuFCLvfSeJw?e=YuPqTs"><span class=MsoSmartlink><
    Reporte de Consumos y Precios.xlsx</a>
    
    <p>Cordial saludo,
    <br>JuanL</p>
    """

In [3]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'jcleiva@grupo-exito.com'
mail.Subject = "Test de correo"
att1=mail.Attachments.Add(r"C:\Users\jcleiva\OneDrive - Grupo-exito.com\Escritorio\Proyectos\Reportes\Imagenes\Consumos.jpg")
att1.PropertyAccessor.SetProperty("http://schemas.microsoft.com/mapi/proptag/0x3712001F", "MyId1")
mail.HTMLBody = html.format(1.1,"MyId1")
#mail.HTMLBody = "<html><body>Test image <img src=""cid:MyId1""></body></html>"
mail.Send()